<a href="https://colab.research.google.com/github/itnote/colaborator/blob/master/ok1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from google.colab import drive
import pandas as pd
import json

mydate = pd.read_excel('https://github.com/itnote/colaborator/blob/master/data/6500000_%EC%A0%9C%EC%A3%BC%ED%8A%B9%EB%B3%84%EC%9E%90%EC%B9%98%EB%8F%84_07_23_02_P_%EC%9C%A0%ED%9D%A5%EC%A3%BC%EC%A0%90%EC%98%81%EC%97%85.xlsx?raw=true')


In [3]:
 

mydate.head()

,번호,개방서비스명,개방서비스ID,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지전화,소재지면적,소재지우편번호,소재지전체주소,도로명전체주소,도로명우편번호,사업장명,최종수정시점,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),위생업태명,남성종사자수,여성종사자수,영업장주변구분명,등급구분명,급수시설구분명,총종업원수,본사종업원수,공장사무직종업원수,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
0,1,유흥주점영업,07_23_02_P,6510000,6510000-102-1990-00001,19900110,NaN,3,폐업,2,폐업,20180320.0,NaN,NaN,NaN,006407467210,246.00,690813.0,제주특별자치도 제주시 연동 268-4번지,제주특별자치도 제주시 신광로 96-1 (연동),63137.0,스윙,20180320111803,I,2018-08-31 23:59:59.0,기타,152489.637380,-1321.301629,기타,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,246.00,NaN,NaN,NaN
1,2,유흥주점영업,07_23_02_P,6510000,6510000-102-1990-00028,19900829,NaN,3,폐업,2,폐업,19980824.0,NaN,NaN,NaN,006407229100,627.10,690836.0,제주특별자치도 제주시 일도일동 1305번지,NaN,NaN,오투,20020131000000,I,2018-08-31 23:59:59.0,카바레,155925.640884,2123.540667,카바레,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,627.10,NaN,NaN,NaN
2,3,유흥주점영업,07_23_02_P,6510000,6510000-102-1985-00001,19850209,NaN,3,폐업,2,폐업,20090918.0,NaN,NaN,NaN,064 7421411,223.00,690813.0,제주특별자치도 제주시 연동 260-34번지 (삼무서2로14),NaN,NaN,애술의전당,20090916120423,I,2018-08-31 23:59:59.0,룸살롱,152463.655857,-508.584395,룸살롱,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,223.00,NaN,NaN,NaN
3,4,유흥주점영업,07_23_02_P,6510000,6510000-102-2007-00047,20071220,NaN,3,폐업,2,폐업,20150416.0,NaN,NaN,NaN,006407450447,115.98,690814.0,제주특별자치도 제주시 연동 274-31번지 (1층),"제주특별자치도 제주시 연동8길 13 (연동,(1층))",63133.0,플라워,20120914123632,I,2018-08-31 23:59:59.0,룸살롱,152597.708291,-1191.781719,룸살롱,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,115.98,NaN,NaN,NaN
4,5,유흥주점영업,07_23_02_P,6510000,6510000-102-1985-00087,19851010,NaN,3,폐업,2,폐업,20170324.0,NaN,NaN,NaN,006407466955,113.68,690813.0,제주특별자치도 제주시 연동 271-36번지,제주특별자치도 제주시 삼무로 지하 41 (연동),NaN,더썸(The some),20170324144146,I,2018-08-31 23:59:59.0,룸살롱,152642.532913,-570.454445,룸살롱,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,113.68,NaN,NaN,NaN


In [0]:
ratings = pd.read_csv('./gdrive/My Drive/_AI/ratings_small.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [0]:
ratings.describe()

,userId,movieId,rating
count,100004.000000,100004.000000,100004.000000
mean,347.011310,12548.664363,3.543608
std,195.163838,26369.198969,1.058064
min,1.000000,1.000000,0.500000
25%,182.000000,1028.000000,3.000000
50%,367.000000,2406.500000,4.000000
75%,520.000000,5418.000000,4.000000
max,671.000000,163949.000000,5.000000


In [0]:

meta.movieId = pd.to_numeric(meta.movieId, errors='coerce')
ratings.movieId = pd.to_numeric(ratings.movieId, errors='coerce')

In [0]:
def parse_genres(genres_str):
    genres = json.loads(genres_str.replace('\'', '"'))
    
    genres_list = []
    for g in genres:
        genres_list.append(g['name'])

    return genres_list

meta['genres'] = meta['genres'].apply(parse_genres)

meta.head()

,movieId,original_title,original_language,genres
0,862,Toy Story,en,"[Animation, Comedy, Family]"
1,8844,Jumanji,en,"[Adventure, Fantasy, Family]"
2,15602,Grumpier Old Men,en,"[Romance, Comedy]"
3,31357,Waiting to Exhale,en,"[Comedy, Drama, Romance]"
4,11862,Father of the Bride Part II,en,[Comedy]


In [0]:
data = pd.merge(ratings, meta, on='movieId', how='inner')

data.head(100)

,userId,movieId,rating,original_title,original_language,genres
0,1,1371,2.5,Rocky III,en,[Drama]
1,4,1371,4.0,Rocky III,en,[Drama]
2,7,1371,3.0,Rocky III,en,[Drama]
3,19,1371,4.0,Rocky III,en,[Drama]
4,21,1371,3.0,Rocky III,en,[Drama]
...,...,...,...,...,...,...
95,15,2105,4.0,American Pie,en,"[Comedy, Romance]"
96,30,2105,2.0,American Pie,en,"[Comedy, Romance]"
97,34,2105,4.0,American Pie,en,"[Comedy, Romance]"
98,35,2105,3.5,American Pie,en,"[Comedy, Romance]"


In [0]:
matrix = data.pivot_table(index='userId', columns='original_title', values='rating')

matrix.head(10)

original_title,10 Things I Hate About You,11'09''01 - September 11,12 Angry Men,15 Minutes,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2010,21 Grams,24 Hour Party People,25th Hour,28 Days Later,29th Street,3 Ninjas: High Noon at Mega Mountain,40 Days and 40 Nights,42nd Street,48 Hrs.,5 Card Stud,50 First Dates,54,8 Mile,A Beautiful Mind,A Bridge Too Far,A Brief History of Time,A Chorus Line,A Christmas Carol,A Christmas Story,A Clockwork Orange,A Close Shave,A Countess from Hong Kong,A Few Good Men,A Hard Day's Night,A Nightmare on Elm Street,A Perfect Murder,A Pyromaniac's Love Story,A River Runs Through It,A Shot in the Dark,A Star Is Born,A Streetcar Named Desire,A Time to Kill,A View to a Kill,...,Voyage to the Bottom of the Sea,Wag the Dog,Wayne's World,Wayne's World 2,We're No Angels,Weekend at Bernie's,"Welcome Home, Roxy Carmichael",Welcome to Woop Woop,Westworld,Wet Hot American Summer,Whale Rider,What Lies Beneath,What Women Want,What's Eating Gilbert Grape,What's New Pussycat?,When Harry Met Sally...,While You Were Sleeping,White Lightning,Wild Orchid,Wild Things,Wild Wild West,Wild at Heart,Wild in the Country,Willy Wonka & the Chocolate Factory,Wish You Were Here,Working Girl,Wuthering Heights,X: The Unknown,Yankee Doodle Dandy,You Only Live Twice,You're a Big Boy Now,Young Adam,Young Black Stallion,Young Frankenstein,Young and Innocent,Zabriskie Point,Zardoz,eXistenZ,xXx,¡Three Amigos!
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,

In [0]:
def pearsonR(s1, s2):

    # print('pearsonRRR')
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()

    retvalue =  np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))
    return retvalue

def recommend(input_movie, matrix, n, similar_genre=True):

    print('Start------' + input_movie)

    input_genres = meta[meta['original_title'] == input_movie]['genres'].iloc(0)[0]
 
    #print( input_genres)
    #print("dddd")
    result = []
    for title in matrix.columns:
        if title == input_movie:
            continue

        # rating comparison

        #print("cccc")
        #print( matrix[input_movie] )
        cor = pearsonR(matrix[input_movie], matrix[title])

        # genre comparison
        if similar_genre and len(input_genres) > 0:
            temp_genres = meta[meta['original_title'] == title]['genres'].iloc(0)[0]

            same_count = np.sum(np.isin(input_genres, temp_genres))
            cor += (GENRE_WEIGHT * same_count)

        if np.isnan(cor):
            continue
        else:
            result.append((title, '{:.2f}'.format(cor), temp_genres))

    result.sort(key=lambda r: r[1], reverse=True)

    return result[:n]

#Prediction

In [0]:
# meta[meta['original_title'] == 'Toy Story']['genres'].iloc(0)[0]
mytitle ='2010'
meta_test = meta[meta['original_title'] == mytitle]
meta_test.head()


,movieId,original_title,original_language,genres
2198,4437,2010,en,"[Thriller, Science Fiction]"


In [0]:
input_genres = meta[meta['original_title'] == mytitle]['genres'].iloc(0)[0]
print(input_genres)

['Thriller', 'Science Fiction']


In [0]:
# matrix.head(10)
print(matrix[mytitle])
 

userId
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
       ..
667   NaN
668   NaN
669   NaN
670   NaN
671   NaN
Name: 2010, Length: 670, dtype: float64


In [0]:
 print(mytitle)
recommend_result = recommend( mytitle , matrix, 10, similar_genre=True) 
pd.DataFrame(recommend_result, columns = ['Title', 'Correlation', 'Genre'])

2010
Start------2010
['Thriller', 'Science Fiction']
dddd
cccc
userId
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
       ..
667   NaN
668   NaN
669   NaN
670   NaN
671   NaN
Name: 2010, Length: 670, dtype: float64
pearsonRRR
cccc
userId
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
       ..
667   NaN
668   NaN
669   NaN
670   NaN
671   NaN
Name: 2010, Length: 670, dtype: float64
pearsonRRR
cccc
userId
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
       ..
667   NaN
668   NaN
669   NaN
670   NaN
671   NaN
Name: 2010, Length: 670, dtype: float64
pearsonRRR
cccc
userId
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
       ..
667   NaN
668   NaN
669   NaN
670   NaN
671   NaN
Name: 2010, Length: 670, dtype: float64
pearsonRRR
cccc
userId
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
       ..
667   NaN
668   NaN
669   NaN
670   NaN
671   NaN
Name: 2010, Length: 670, dtype: float64
pearsonRRR
cccc
userId
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
       ..
667  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


cccc
userId
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
       ..
667   NaN
668   NaN
669   NaN
670   NaN
671   NaN
Name: 2010, Length: 670, dtype: float64
pearsonRRR
cccc
userId
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
       ..
667   NaN
668   NaN
669   NaN
670   NaN
671   NaN
Name: 2010, Length: 670, dtype: float64
pearsonRRR
cccc
userId
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
       ..
667   NaN
668   NaN
669   NaN
670   NaN
671   NaN
Name: 2010, Length: 670, dtype: float64
pearsonRRR
cccc
userId
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
       ..
667   NaN
668   NaN
669   NaN
670   NaN
671   NaN
Name: 2010, Length: 670, dtype: float64
pearsonRRR
cccc
userId
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
       ..
667   NaN
668   NaN
669   NaN
670   NaN
671   NaN
Name: 2010, Length: 670, dtype: float64
pearsonRRR
cccc
userId
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
       ..
667   NaN
668   NaN
669   NaN
670   NaN
671   NaN
Name: 2010, L